In [1]:
import os

from langchain_openai import OpenAI, OpenAIEmbeddings
from pinecone import Pinecone

In [2]:
%pwd

'/home/rohwid/GitHub/question-and-answer-chatbot/notebooks'

In [3]:
# Change to the main directory
# So, it's executed from main directory
os.chdir("../")

In [4]:
%pwd

'/home/rohwid/GitHub/question-and-answer-chatbot'

In [5]:
with open('.creds') as f:
    os.environ.update(
        line.strip().split('=') for line in f
)

In [6]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
PINECONE_INDEX = os.environ.get('PINECONE_INDEX')

In [7]:
pincone = Pinecone(api_key=PINECONE_API_KEY)
index = pincone.Index(PINECONE_INDEX)

In [8]:
from langchain_pinecone import Pinecone

In [9]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=1536)
text_field = "text"

query = "What are the specific features or aspects that users appreciate the most in our application?"

vectorstore = Pinecone(
    index, embeddings, text_field, namespace='ns0'
)

In [10]:
vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

[Document(page_content='review_text: Awesome UI, best music app out there!', metadata={'author_app_version': '1.1.0.91', 'author_name': 'A Google user', 'pseudo_author_id': '286593453219054880269', 'review_id': '672a155a-e81e-4d28-bdeb-a74c031bc072', 'review_likes': '1', 'review_rating': '5', 'review_timestamp': '2014-05-27 15:17:20', 'row': 3.0, 'source': '672a155a-e81e-4d28-bdeb-a74c031bc072'}),
 Document(page_content='review_text: Just started using this app n loving it already  Keep up the great work', metadata={'author_app_version': '1.1.0.112', 'author_name': 'A Google user', 'pseudo_author_id': '264385232751371829139', 'review_id': 'f4e63586-3b96-4789-ba43-b571f90a8ea5', 'review_likes': '0', 'review_rating': '4', 'review_timestamp': '2014-05-27 22:21:41', 'row': 48.0, 'source': 'f4e63586-3b96-4789-ba43-b571f90a8ea5'}),
 Document(page_content='review_text: I enjoy the awesome UI of this app, and it has all the music one can ask for.', metadata={'author_app_version': '1.1.0.91', '

In [11]:
from langchain.chains import RetrievalQA

In [12]:
llm = OpenAI(temperature=0, model_name='gpt-3.5-turbo-instruct', openai_api_key=OPENAI_API_KEY)

In [13]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

qa_result = qa.invoke(query)

## Summarization with Prompt

Call the `PromptTemplate` module from Langchain to setup the instruction the model to perform summarization.

In [14]:
from langchain import PromptTemplate

In [15]:
template = """
%INSTRUCTIONS:
Please summarize the following piece of text.
Convert the size metric to centimeters if exist.

%TEXT:
{text}
"""

# Create a LangChain prompt template that we can insert values to later
prompt = PromptTemplate(
    input_variables=["text"],
    template=template,
)

Also take the search result from PDF content and Google with SerpAPI. The output from the PDF file and also Google search result will be make the OpenAI know more about the current product that officialy released in February 2023.

In [17]:
confusing_text = """{from_qna}
""".format(
    from_qna = qa_result['result'].strip()
)

print(confusing_text)

Based on the given review texts, users appreciate the UI (user interface) and the music selection in the application. They also seem to appreciate any updates or improvements made to the design of the app.



Here are the final prompt that contain instruction and text that need to be summarized.

In [18]:
summarize_prompt = prompt.format(text=confusing_text)
print(summarize_prompt)


%INSTRUCTIONS:
Please summarize the following piece of text.
Convert the size metric to centimeters if exist.

%TEXT:
Based on the given review texts, users appreciate the UI (user interface) and the music selection in the application. They also seem to appreciate any updates or improvements made to the design of the app.




Perform summarization with OpenAI `text-davinci-003` model.

In [21]:
summerize = OpenAI(temperature=0, model_name='gpt-3.5-turbo-instruct', openai_api_key=OPENAI_API_KEY)
summarize_result = summerize.invoke(summarize_prompt)
print(summarize_result)


Users like the UI and music selection in the app, and appreciate any design updates or improvements. Convert size to centimeters if available.
